In [1]:
from collections import defaultdict, namedtuple
from importlib import reload
from pathlib import Path
from string import ascii_lowercase as LETTERS
from random import sample, choice, uniform
from time import time
from math import log, exp
import codecs
import csv
import logging
import random
import sys
import urllib.request
import zipfile

from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd

project_dir = Path.cwd().parents[0]
sys.path.insert(0, str(project_dir / 'src')) 
import crypto as cg; reload(cg)

<module 'crypto' from '/Users/chris/Documents/cryptogram-solver/src/crypto.py'>

In [2]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [3]:
def get_documents(max_rows=250):
    csv.field_size_limit(sys.maxsize)
    articles = list()
    for article_num in range(1, 4):
        zipname = Path('..', 'data', 'raw', f'articles{article_num}.csv.zip')
        filename = f'articles{article_num}.csv'
        with zipfile.ZipFile(zipname) as z:
            with z.open(filename, 'r') as f:
                csvfile = csv.reader(codecs.iterdecode(f, 'utf-8'))
                for row, line in enumerate(csvfile):
                    articles.append(line[-1])  # get content column
                    if row == max_rows:  # per file
                        break
    articles = articles[1:]  # remove column header
    return articles

In [4]:
docs = get_documents()

In [786]:
# news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=20180827)
# news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=20180827)

In [5]:
tokenizer_test = cg.Tokenizer()
tokenizer_test.tokenize("This is a sentence.")

{Token(ngrams=('this',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('is',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('a',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('sentence',), kind=<NgramKind: word>, n=1): 1,
 Token(ngrams=('<',), kind=<NgramKind: char>, n=1): 4,
 Token(ngrams=('t',), kind=<NgramKind: char>, n=1): 2,
 Token(ngrams=('h',), kind=<NgramKind: char>, n=1): 1,
 Token(ngrams=('i',), kind=<NgramKind: char>, n=1): 2,
 Token(ngrams=('s',), kind=<NgramKind: char>, n=1): 3,
 Token(ngrams=('>',), kind=<NgramKind: char>, n=1): 4,
 Token(ngrams=('<', 't'), kind=<NgramKind: char>, n=2): 1,
 Token(ngrams=('t', 'h'), kind=<NgramKind: char>, n=2): 1,
 Token(ngrams=('h', 'i'), kind=<NgramKind: char>, n=2): 1,
 Token(ngrams=('i', 's'), kind=<NgramKind: char>, n=2): 2,
 Token(ngrams=('s', '>'), kind=<NgramKind: char>, n=2): 2,
 Token(ngrams=('<', 't', 'h'), kind=<NgramKind: char>, n=3): 1,
 Token(ngrams=('t', 'h', 'i'), kind=<NgramKind: char>, n=3): 1,
 Token(ngrams=('

In [6]:
reload(cg)
tokenizer = cg.Tokenizer(char_ngram_range=(1, 3), word_ngram_range=(1, 1), vocab_size=1000000)
# tokenizer.fit(news_train['data'][:2500])
tokenizer.fit(docs[:250])
print(len(tokenizer.vocab))
print(tokenizer.totals)

100%|██████████| 250/250 [00:10<00:00, 24.11it/s]

26703
{(<NgramKind: word>, 1): 279190, (<NgramKind: char>, 1): 1873281, (<NgramKind: char>, 2): 1594091, (<NgramKind: char>, 3): 1314901}


In [7]:
tokenizer.totals

{(<NgramKind: word>, 1): 279190,
 (<NgramKind: char>, 1): 1873281,
 (<NgramKind: char>, 2): 1594091,
 (<NgramKind: char>, 3): 1314901}

In [8]:
# doc = cg.Doc(news_test['data'][0].lower())
doc = cg.Doc(docs[0].lower())
mapper = cg.Mapping()
mapper.scramble()
doc = mapper.translate(doc)
doc

'ldkpvxiejx  —   rjxigzkkvjxdc gzotycvrdxk pdnz d xzl fzdg lpzx ve rjqzk ej epzvg    pzdcep rdgz cdlktve didvxke epz jydqd duqvxvkegdevjx: epzb qvipe lvx. epz vxrjqvxi egtqo duqvxvkegdevjx rjtcu rpjjkz ej xj cjxizg uzfzxu epz zmzrtevnz ygdxrp didvxke epz ktve, lpvrp rpdcczxizk epz duqvxvkegdevjx’k dtepjgveb ej kozxu yvccvjxk jf ujccdgk jx pzdcep vxktgdxrz ktykvuvzk fjg   dxu   dqzgvrdxk, pdxuvxi pjtkz gzotycvrdxk d yvi nvrejgb jx    vkktzk. yte d ktuuzx cjkk jf epz uvkotezu ktykvuvzk rjtcu rjxrzvndycb rdtkz epz pzdcep rdgz ogjigdq ej vqocjuz, czdnvxi qvccvjxk jf ozjocz lvepjte drrzkk ej pzdcep vxktgdxrz yzfjgz gzotycvrdxk pdnz ogzodgzu d gzocdrzqzxe. epde rjtcu czdu ej rpdjk vx epz vxktgdxrz qdghze dxu kotg d ojcvevrdc ydrhcdkp wtke dk gzotycvrdxk idvx ftcc rjxegjc jf epz ijnzgxqzxe. ej kednz jff epde jterjqz, gzotycvrdxk rjtcu fvxu epzqkzcnzk vx epz dlhldgu ojkvevjx jf doogjogvdevxi ptiz ktqk ej ezqojgdgvcb ogjo to epz jydqd pzdcep rdgz cdl, dxizgvxi rjxkzgndevnz njezgk lpj pdnz yzzx 

In [791]:
[print(f'{x:0.3g}') for x in np.exp(np.linspace(3, -4, 5))];

20.1
3.49
0.607
0.105
0.0183


In [792]:
solver = cg.Solver(tokenizer, None, 0.1)

In [67]:
i1, i2 = sample(range(len(LETTERS)), 2)
self.key[i1], self.key[i2] = self.key[i2], self.key[i1]

(10, 9)

In [79]:
# class Mapping:
#     def __init__(self, key=None):
#         self.key = key or LETTERS

#     def scramble(self):
#         self.key = ''.join(sample(self.key, len(self.key)))

#     def swap(self, l1, l2, inplace=False):
#         tmp = '_'
#         key = self.key\
#             .replace(l1, tmp)\
#             .replace(l2, l1)\
#             .replace(tmp, l2)
#         if inplace:
#             self.key = key
#         else:
#             return Mapping(key)

#     # def random_swap(self, letters=None, inplace=False):
#     #     letters = letters or LETTERS
#     #     l1 = sample(letters, 1)[0]
#     #     l2 = sample(set(LETTERS) - set(l1), 1)[0]
#     #     return self.swap(l1, l2, inplace)
    
#     def random_swap(self, n, inplace=False):
#         for _ in range(n):
#             i1, i2 = sample(range(len(LETTERS)), 2)
#             self.key = list(self.key)
#             self.key[i1], self.key[i2] = self.key[i2], self.key[i1]
#             self.key = ''.join(self.key)

#     def translate(self, text):
#         trans = str.maketrans(self.key, LETTERS)
#         return type(text)(text.translate(trans))
    


In [205]:
mapping = Mapping()
mapping.random_swap(5)
mapping.key

'abqdefgmojklhnizcpstuvwxyr'

In [823]:
'''
Ideas:
> Implement simulated annealing properly (single swap and decision), with a reasonable temp scheduler.
  - This works! But need to play with temperature scheduler since results are sensitive to even small changes.
> Implement simulated annealing but using softmax and subsetting swap options.
> More swaps in the beginning, fewer later.
'''

def simulated_annealing(text, solver, tokenizer, num_epochs=10000, is_debug=True):
    logger = logging.getLogger(__name__)
    best_mapping = mapping = cg.Mapping()
    doc = cg.Doc(text)
    best_score = solver.score(doc)
    epoch = 0
    decisions = defaultdict(int)
    temps = np.exp(np.linspace(0, -6, num_epochs))
    decisions['good'], decisions['bad_keep'], decisions['bad_pass'] = 0, 0, 0
    for temp in tqdm(temps):
        improving = False
        new_mapping = mapping.random_swap(doc.letters)
        new_doc = new_mapping.translate(doc)
        score = solver.score(new_doc)
        score_change = score - best_score
        if score_change < 0:
            # logger.debug('updating (improvement)')
            best_mapping = new_mapping
            best_score = score
            decisions['good'] += 1
        elif exp(-score_change / temp) > uniform(0, 1):
            # Break this out as different section just for debugging.
            # logger.debug(f'updating (bad change): {score_change}')
            best_mapping = new_mapping
            best_score = score
            decisions['bad_keep'] += 1
        else:
            decisions['bad_pass'] += 1
            # logger.debug(f'keeping: {score_change}')
        mapping = best_mapping
        epoch += 1
        if epoch % 1000 == 0:
            logger.debug(f'{score:0.5g}, {mapping.mapping}, {mapping.translate(doc).text}')
            logger.debug(sorted(list(decisions.items())))
            logger.debug(pd.DataFrame(sorted(list(decisions.items()))))
            decisions = defaultdict(int)
    logger.info(f'\nfinal best ({epoch} epochs): {best_score:0.5g}')
    return mapping.translate(doc).text

In [794]:
# doc = cg.Doc(doc.lower())

In [822]:
# [print(f'{x:0.3g}') for x in np.exp(np.linspace(-30, -5, 100))];

In [814]:
# This is slower than the code above, and less consistent. But it does work.

def choose_new_mapping(doc, mapping, score, proportion, temp):
    
    swap_options = cg.get_swap_options(doc.letters, p=proportion)
    mapping_options = [mapping.swap(*s) for s in swap_options]
    scores = [solver.score(m.translate(doc)) for m in mapping_options]
    
    # Add current without having to recompute score.
    mapping_options.append(mapping)
    scores.append(score)
    
    probs = solver.softmax(-np.array(scores), temp)
    # print(max(probs))
    index = np.random.choice(range(len(probs)), size=1, p=probs)[0]
    
    return mapping_options[index], scores[index]


def simulated_annealing(text, solver, tokenizer, epochs, epochsbug=True):
    logger = logging.getLogger(__name__)
    best_mapping = mapping = cg.Mapping()
    doc = cg.Doc(text)
    epoch = 0
    score = solver.score(mapping.translate(doc))
    temps = np.exp(np.linspace(0, -6, epochs))
    proportions = np.exp(np.linspace(-30, -5, epochs))
    for temp, proportion in tqdm(zip(temps, proportions)):
        mapping, score = choose_new_mapping(doc, mapping, score, proportion, temp)
        epoch += 1
        if epoch % 250 == 0:
            logger.debug((
                f'{score:0.5g}, {mapping.mapping}, {mapping.translate(doc).text} '
                f'temp: {temp:0.5g}, proportion: {proportion:0.5g}'
            ))
    logger.info(f'\nfinal best ({epoch} epochs): {epoch:0.5g}')
    return mapping.translate(doc).text

In [815]:
import spacy
nlp = spacy.load('en')

In [816]:
# text = doc.text
# text = 'Rbo rpktigo vcrb bwucja wj kloj hcjd, km sktpqo, cq rbwr loklgo vcgg cjqcqr kj skhcja wgkja wjd rpycja rk ltr rbcjaq cj cr. -- Roppy Lpwrsborr'
text = 'This is the story of a girl. Who cried a river and drowned the whole world. And while she looked so sad in photographs, I absolutely love her when she smiles.'
# text = 'Jail zesty vixen who grabbed pay from quack.'
# text = "I've found that when everyone rallies behind a cause, and when they learn their effort can contribute something bigger, they get engaged."
# text = str(random.choice(list(nlp(random.choice(articles)).sents)))
doc = cg.Doc(text.lower())
mapping = cg.Mapping()
mapping.scramble()
doc = mapping.translate(doc)
doc.text

'bvgi gi bvk ibawu ap s ogwz. lva mwgkx s wgekw sfx xwalfkx bvk lvazk lawzx. sfx lvgzk ivk zaahkx ia isx gf jvabaowsjvi, g sciaznbkzu zaek vkw lvkf ivk itgzki.'

In [817]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
logger.debug('hi')

2018-09-18 00:55:43 DEBUG    hi


In [831]:
timer = cg.Timer()
timer.tic()
logging.basicConfig(level=logging.DEBUG)
text = simulated_annealing(doc.text, solver, tokenizer, 10000)
print(text)
print(timer.toc())

  9%|▉         | 945/10000 [00:01<00:12, 717.62it/s]2018-09-18 00:58:56 DEBUG    15.88, rdvkzyqsbfjamphocuxiwtgnle, icwt wt icd tilur ln h pwue. ycl muwds h uwzdu hjs sulyjds icd ycled ylues. hjs ycwed tcd ellods tl ths wj kclilpuhkct, w hqtlexider elzd cdu ycdj tcd tvwedt.
2018-09-18 00:58:56 DEBUG    [('bad_keep', 416), ('bad_pass', 184), ('good', 400)]
2018-09-18 00:58:56 DEBUG              0    1
0  bad_keep  416
1  bad_pass  184
2      good  400
 19%|█▉        | 1932/10000 [00:02<00:11, 728.50it/s]2018-09-18 00:58:58 DEBUG    15.478, kcjfilhzxqgwveadnuryptbsom, wmke ke wma ewolr ou x yklh. fmo zlkai x lknal xdi ilofdai wma fmoha folhi. xdi fmkha ema hoogai eo exi kd cmowoylxcme, k xbeohqwahr hona mal fmad ema evkhae.
2018-09-18 00:58:58 DEBUG    [('bad_keep', 349), ('bad_pass', 314), ('good', 337)]
2018-09-18 00:58:58 DEBUG              0    1
0  bad_keep  349
1  bad_pass  314
2      good  337
 30%|██▉       | 2979/10000 [00:04<00:09, 716.89it/s]2018-09-18 00:58:59 DEBUG    14.861

hira ra his ahnom ng l prou. win forst l ordso let tonwest his winus wnout. let wirus ais unnkst an alt re vinhnpolvia, r lyanuchsum unds iso wise ais abrusa.
14.225373029708862
